In [ ]:
from speem.instruments.motors.drivers import (
    Automation1Driver,
    MdriveDriver,
    CONTROLLER,
    MDRIVE_COMM,
)
from speem.instruments.motors.specs import RealAxis, REAL_AXIS_SPECS
from pymdrive import emergency_stop

: 

In [2]:
runtime = CONTROLLER.runtime
await MDRIVE_COMM.start()

x = Automation1Driver(RealAxis.x, **REAL_AXIS_SPECS[RealAxis.x])
y = Automation1Driver(RealAxis.y, **REAL_AXIS_SPECS[RealAxis.y])
Z = Automation1Driver(RealAxis.Z, **REAL_AXIS_SPECS[RealAxis.Z])

X = MdriveDriver(RealAxis.X, **REAL_AXIS_SPECS[RealAxis.X])
Y = MdriveDriver(RealAxis.Y, **REAL_AXIS_SPECS[RealAxis.Y])

Theta = Automation1Driver(RealAxis.Theta, **REAL_AXIS_SPECS[RealAxis.Theta])

In [5]:
await MDRIVE_COMM.stop()

In [7]:
X.enable()
await X.get_position()

0.0

In [8]:
print(await X.is_homed())
print(await Y.is_homed())

False
False


In [3]:
Y.enable()
await Y.get_position()

-50.809999984459864

In [8]:
await X.move_relative(2)

In [61]:
await x.home()

In [119]:
await x.move_relative(-0.1)

In [67]:
await y.move_relative(18.5)

In [3]:
await Y.home()

In [4]:
await Y.move_absolute(0)

moving real Y to 0


In [94]:
await Y.move_relative(-0.1)

In [95]:
print(await Y.get_position())
Y.soft_limits

-45.52002086600077


(-50, 0)

In [114]:
Theta.enable()
await Theta.get_position()

197.99999999999997

In [113]:
await Theta.move_relative(0.5)

In [5]:
print(await x.get_position())
print(x.soft_limits)
print(x.faults)
print(await x.is_moving())

0.00171
(-5.0, 0.1)
[]
False


In [55]:
await X.get_position()

20.499996444868167

In [54]:
await X.move_relative(0.5)

In [48]:
soft_limits = x.soft_limits
print(soft_limits[0] * x.unit_ratio, soft_limits[1] * x.unit_ratio)

-10000.0 200.0


In [ ]:
import serial, sys, os, psutil


def kill_process_using_port(port):
    """Force kill any process using the specified port."""
    try:
        # Get list of all running processes
        for proc in psutil.process_iter(["pid", "name"]):
            try:
                # Get connections opened by the process
                connections = proc.net_connections()
                for conn in connections():
                    if conn.laddr.port == int(port.replace("COM", "")):
                        print(
                            f"Killing process {proc.info['name']} (PID: {proc.info['pid']})"
                        )
                        proc.terminate()
                        proc.wait(timeout=3)
                        proc.kill()
            except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
                pass
    except Exception as e:
        print(f"Error killing processes: {e}")


def force_open_serial_port(port="COM3", baudrate=9600):
    """Force close and reopen a serial port."""
    try:
        # First, kill any processes using the port
        kill_process_using_port(port)

        # Wait briefly to ensure processes are terminated
        import time

        time.sleep(1)

        # Try to open the port
        ser = serial.Serial(port=port, baudrate=baudrate, timeout=1)
        print(f"Successfully opened {port}")
        return ser

    except serial.SerialException as e:
        print(f"Failed to open {port}: {str(e)}")
        return None


# Example usage
port = "COM3"
ser = force_open_serial_port(port)

if ser:
    try:
        # Your serial operations here
        pass
    finally:
        ser.close()

In [18]:
import serial, psutil
from collections import Counter

from speem.instruments.motors.drivers import MDRIVE_COMM

all_ports = []
for process in psutil.process_iter(["pid", "name"]):
    try:
        connections = process.net_connections()
        all_ports.extend([connection.laddr.port for connection in connections])
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

try:
    serial.Serial("COM3")
except Exception:
    print("failed")
del MDRIVE_COMM
try:
    serial.Serial("COM3")
except Exception:
    print("failed")

no_mdrive_ports = []
for process in psutil.process_iter(["pid", "name"]):
    try:
        connections = process.net_connections()
        no_mdrive_ports.extend([connection.laddr.port for connection in connections])
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass


def find_missing_with_duplicates(complete_list, incomplete_list):
    """Find missing values accounting for duplicates"""
    complete_counts = Counter(complete_list)
    incomplete_counts = Counter(incomplete_list)

    missing = []
    for value, complete_count in complete_counts.items():
        incomplete_count = incomplete_counts.get(value, 0)
        if incomplete_count < complete_count:
            # Add the missing occurrences
            missing.extend([value] * (complete_count - incomplete_count))

    return missing


missing = find_missing_with_duplicates(all_ports, no_mdrive_ports)
print(missing)

failed
failed
[]
